<a href="https://colab.research.google.com/github/borbysh/Momentum-QNG/blob/main/Quantum_Machine_Learning/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pennylane

In [11]:
import pennylane as qml
from pennylane import numpy as np

Adopted from this thread: https://discuss.pennylane.ai/t/training-with-qng-optimizer-on-circuit-with-data-argument/2067

In [16]:
Nwires = 5
dev = qml.device("default.qubit", wires=Nwires)

@qml.qnode(dev)
def circuit(params, data):
    qml.AngleEmbedding(data, wires=range(Nwires))
    qml.StronglyEntanglingLayers(params, wires=range(Nwires))
    return qml.expval(qml.PauliZ(Nwires-1))
    #return qml.expval(qml.PauliZ(0))

def cost(params, single_sample):
     return (1 - circuit(params, single_sample)) ** 2

In [17]:
Nbatch = 10
np.random.seed(1)
data = [np.random.random([Nwires], requires_grad=False) for _ in range(Nbatch)]
init_params = np.random.random(qml.StronglyEntanglingLayers.shape(Nwires, Nwires), requires_grad=True)

In [22]:
params = init_params
opt = qml.MomentumQNGOptimizer(stepsize=0.01) #, lam=0.5)
#opt = qml.QNGOptimizer(0.01)
for it in range(20):
    for sample in data:
        cost_fn = lambda p: cost(p, sample)
        metric_fn = lambda p: qml.metric_tensor(circuit, approx="block-diag")(p, sample)

        params, loss = opt.step_and_cost(cost_fn, params,  metric_tensor_fn=metric_fn)

    print(f"Epoch: {it} | Loss: {loss} |")

Epoch: 0 | Loss: 0.15549695541357283 |
Epoch: 1 | Loss: 0.05988706628428329 |
Epoch: 2 | Loss: 0.042608079109815296 |
Epoch: 3 | Loss: 0.009655835195325006 |
Epoch: 4 | Loss: 0.009107617638430017 |
Epoch: 5 | Loss: 0.009786467945918965 |
Epoch: 6 | Loss: 0.00494552775393942 |
Epoch: 7 | Loss: 0.003910207831195334 |
Epoch: 8 | Loss: 0.004644458519323576 |
Epoch: 9 | Loss: 0.004848368295093205 |
Epoch: 10 | Loss: 0.0056210984433385245 |
Epoch: 11 | Loss: 0.0057894144255329185 |
Epoch: 12 | Loss: 0.0022214316292274213 |
Epoch: 13 | Loss: 0.0016241872083111756 |
Epoch: 14 | Loss: 0.0020737342430632005 |
Epoch: 15 | Loss: 0.002011271751105126 |
Epoch: 16 | Loss: 0.0019071500311215004 |
Epoch: 17 | Loss: 0.002128495421098454 |
Epoch: 18 | Loss: 0.002794015627004573 |
Epoch: 19 | Loss: 0.0018101366613166817 |


In [27]:
params = init_params
#opt = qml.MomentumOptimizer(0.25)
opt = qml.AdamOptimizer(0.3)
for it in range(20):
    for sample in data:
        cost_fn = lambda p: cost(p, sample)
        params, loss = opt.step_and_cost(cost_fn, params)

    print(f"Epoch: {it} | Loss: {loss} |")

Epoch: 0 | Loss: 0.04624321895554315 |
Epoch: 1 | Loss: 0.02655710977331679 |
Epoch: 2 | Loss: 0.005047409158737415 |
Epoch: 3 | Loss: 0.00184142298152455 |
Epoch: 4 | Loss: 0.0001323355668437131 |
Epoch: 5 | Loss: 0.0005171355954513288 |
Epoch: 6 | Loss: 0.0014999484135997491 |
Epoch: 7 | Loss: 0.00184955456304657 |
Epoch: 8 | Loss: 0.0028353575518977187 |
Epoch: 9 | Loss: 0.002081845800501092 |
Epoch: 10 | Loss: 0.0005504415210112592 |
Epoch: 11 | Loss: 0.0008673205647957051 |
Epoch: 12 | Loss: 0.000930478095192633 |
Epoch: 13 | Loss: 0.00042083822014304316 |
Epoch: 14 | Loss: 0.0007118981484338083 |
Epoch: 15 | Loss: 0.0006118166049433029 |
Epoch: 16 | Loss: 0.0006835555729658541 |
Epoch: 17 | Loss: 0.0007129300866910232 |
Epoch: 18 | Loss: 0.0008971942210156729 |
Epoch: 19 | Loss: 0.000873066604117647 |


In [34]:
def cost_batch(params, data):
     return np.sum((1 - circuit(params, single_sample)) ** 2 for single_sample in data) / Nbatch

In [35]:
params = init_params
opt = qml.MomentumQNGOptimizer(stepsize=0.01) #, lam=0.5)
#opt = qml.QNGOptimizer(0.04)
for it in range(200):
    metric_fn = lambda p: np.sum((qml.metric_tensor(circuit, approx="block-diag")(p, sample) for sample in data), axis = 0)/Nbatch
    cost_fn = lambda p: cost_batch(p, data)
    params, loss = opt.step_and_cost(cost_fn, params,  metric_tensor_fn=metric_fn)

    print(f"Epoch: {it} | Loss: {loss} |")

Epoch: 0 | Loss: 0.8005877824610932 |


KeyboardInterrupt: 

In [36]:
params = init_params
opt = qml.MomentumOptimizer(stepsize=0.05) #, lam=0.5)
#opt = qml.AdamOptimizer(0.4)
for it in range(200):
    cost_fn = lambda p: cost_batch(p, data)
    params, loss = opt.step_and_cost(cost_fn, params)

    print(f"Epoch: {it} | Loss: {loss} |")

Epoch: 0 | Loss: 0.8005877824610932 |
Epoch: 1 | Loss: 0.689309159850268 |
Epoch: 2 | Loss: 0.5128848848516961 |
Epoch: 3 | Loss: 0.33586376643435706 |
Epoch: 4 | Loss: 0.20717285630028007 |
Epoch: 5 | Loss: 0.13719511720888644 |
Epoch: 6 | Loss: 0.11151536001804185 |
Epoch: 7 | Loss: 0.1134018817221522 |


KeyboardInterrupt: 